# Using Tools

In [14]:
import urllib.request
import json
import urllib.parse
from datetime import date
# Define the API key and endpoint
api_key = "52DAJTUQN4SUMNDW7WTEVJDDN"
#location = "New York, NY"  # Example location, replace with your desired one
base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"

# Set the parameters
params = f"?unitGroup=metric&key={api_key}&contentType=json&include=current&elements=datetime,datetimeEpoch,name,temp,humidity&options=stnslevel1"

# Assemble the full request URL
def get_current_weather(location):

    try:
        # Make the request to the Weather API
        location_encoded = urllib.parse.quote_plus(location)
        request_url = f"{base_url}{location_encoded}/{date.today().isoformat()}{params}"

        response = urllib.request.urlopen(request_url)
        data = response.read()
        
        # Parse the JSON data
        weather_data = json.loads(data)
        
        # Print some of the fetched weather data
        result = f"Weather data for {location}:"
        #print(f"Weather data for {location}:")
        for day_data in weather_data['days']:
            result += "\n" + f"Date: {day_data['datetime']}, Temperature: {day_data['temp']*10}°C, Humidity: {day_data['humidity']}%"
        print("get_current_weather")
    except urllib.error.HTTPError as e:
        # Print error message if the request fails
        error_info = e.read().decode()
        print('HTTP Error:', e.code, error_info)

    except urllib.error.URLError as e:
        # Print error message if there is a URL error
        print('URL Error:', e.reason)
    return result

In [34]:
def get_score(name):
    mapping = {
            "sam": "400",
            "mark": "480",
            "john":"500",
            "frank":"450"
        }
    print(f"getting score for {name}")
    return mapping.get(name)

In [ ]:
def validate_name(name):
    locs = ["sam","mark","john","frank"]
    return "valid" if name.lower() in locs else "invalid"

In [18]:
print(get_current_weather("Gurgaon"))

get_current_weather
Weather data for Gurgaon:
Date: 2025-12-29, Temperature: 145.0°C, Humidity: 70.1%


In [20]:
get_score_function = {
    "name": "get_score",
    "description": "Get the student score for the given name",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The student for which score is to be given",
            },
        },
        "required": ["name"],
        "additionalProperties": False
    }
}


validate_name_function = {
    "name": "validate_name",
    "description": "Validate the given student name. For valid name it will return 'valid', else 'invalid'",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The status, 'valid' for correct name else 'invalid'",
            },
        },
        "required": ["name"],
        "additionalProperties": False
    }
}


tools = [{"type": "function", "function": validate_name_function},
         {"type": "function", "function": get_score_function}]

In [21]:
system_prompt = "You are school examiner who is resposibly for giving scores to the student, you have tools to " \
"give the scores. There are 2 tools, one is to validate the student name and other is to get the score for the validated student. Give the score of asked student or say you can not find the score"

In [29]:
def handle_tools_call(message):
    mapping = {
            "get_score": get_score,
            "validate_name": validate_name
        }
    results = []
    i = 1
    print(message.tool_calls)
    for tool_call in message.tool_calls:
        print(f"{i}. in handle_tools_call")
        i += 1
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        name = arguments.get('name')
        tool = mapping.get(tool_name)
        if tool_name == "get_score" :
            result = get_score(name)
        if tool_name == "validate_name" :
            result = validate_name(name)
        results.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
    return results

In [30]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

gemini_key = os.getenv("GEMINI_API_KEY")

def check_weather(location):
    user_prompt = f"Give the scores for both {location}"
    messages = [{"role":"system", "content":system_prompt}, 
                {"role":"user", "content":user_prompt}]
    openAi = OpenAI(api_key=gemini_key,base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
    response = openAi.chat.completions.create( model="gemini-2.5-flash-lite", messages=messages, tools = tools)
    i = 1
    print(response)
    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(f"{i}. in while loop")
        i += 1
        responses = handle_tools_call(message)
        messages.append(message)
        messages.extend(responses)
        response = openAi.chat.completions.create(model="gemini-2.5-flash-lite", messages=messages, tools=tools)
    
    return response.choices[0].message.content


In [36]:
print(check_weather("sam and kim"))

ChatCompletion(id='03tSadmbAbSvjuMP5sGyeQ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='function-call-10185808000155880810', function=Function(arguments='{"name":"sam"}', name='validate_name'), type='function'), ChatCompletionMessageFunctionToolCall(id='function-call-10185808000155879231', function=Function(arguments='{"name":"kim"}', name='validate_name'), type='function')]))], created=1767013331, model='gemini-2.5-flash-lite', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=30, prompt_tokens=184, total_tokens=214, completion_tokens_details=None, prompt_tokens_details=None))
1. in while loop
[ChatCompletionMessageFunctionToolCall(id='function-call-10185808000155880810', function=Function(arguments='{"name":"sam"}', name='va